In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
# mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
# clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
# landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
# landsat_vis = {
#     'min': 0,
#     'max': 1,
#     'bands': ['b5', 'b4', 'b3']
# }
# Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
# Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
# Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
# Map

In [5]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'countries')


In [6]:
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [7]:
work_dir = os.path.expanduser('../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')

In [8]:
start_date = '2020-12-01'
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [9]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map.centerObject(patagonia_grande, 10)
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [10]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [11]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [12]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [13]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [14]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

In [15]:
greenest = withNDVI.qualityMosaic('NDVI').clip(patagonia_grande)

In [16]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

In [17]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [18]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [19]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [20]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# paleta de 18 colores


In [21]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [22]:
landcover_2018.bandNames().getInfo()

['b1']

In [23]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [24]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [25]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [26]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [27]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [28]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [29]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [30]:
# poligono = ee.FeatureCollection([
#     ee.Feature(achaparrado),
#     ee.Feature(agricolas),
#     ee.Feature(agua),
#     ee.Feature(arenalroca),
#     ee.Feature(bosquedegradado),
#     ee.Feature(cuncuna),
#     ee.Feature(estepa),
#     ee.Feature(humedal),
#     ee.Feature(mallin),
#     ee.Feature(matorralabierto),
#     ee.Feature(matdenso),
#     ee.Feature(plantacion1),
#     ee.Feature(plantacion2),
#     ee.Feature(plantacion3),
#     ee.Feature(pradera),
#     ee.Feature(primario),
#     ee.Feature(secundario),
#     ee.Feature(sombra),
#     ee.Feature(suelo),    
# ])


In [31]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [32]:
dataset = greenest.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [33]:
dataset.size().getInfo()

20343

In [34]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.08925435692071915, 'B10': 292.0626220703125, 'B11': 292.143798828125, 'B2': 0.06809869408607483, 'B3': 0.051249273121356964, 'B4': 0.03142053633928299, 'B5': 0.3350105583667755, 'B6': 0.11977358907461166, 'B7': 0.044714801013469696, 'B8': 0.04191073775291443, 'B9': 0.005282651633024216, 'BQA': 2720, 'ID': 1, 'NDVI': 0.8285050988197327, 'date': 20210105, 'doy': 5, 'month': 1}}


In [35]:
# Guardamos las bandas
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
sample = dataset.randomColumn()

split = 0.7 

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [36]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_1', 'properties': {'B1': 0.08922932296991348, 'B10': 291.713134765625, 'B11': 291.7758483886719, 'B2': 0.06894993036985397, 'B3': 0.05325217545032501, 'B4': 0.03204644098877907, 'B5': 0.35243579745292664, 'B6': 0.12600763142108917, 'B7': 0.04679281637072563, 'B8': 0.043412916362285614, 'B9': 0.005157469771802425, 'BQA': 2720, 'ID': 1, 'NDVI': 0.833300769329071, 'date': 20210105, 'doy': 5, 'month': 1, 'random': 0.33402919438639733}}


In [37]:
print(validation.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.08925435692071915, 'B10': 292.0626220703125, 'B11': 292.143798828125, 'B2': 0.06809869408607483, 'B3': 0.051249273121356964, 'B4': 0.03142053633928299, 'B5': 0.3350105583667755, 'B6': 0.11977358907461166, 'B7': 0.044714801013469696, 'B8': 0.04191073775291443, 'B9': 0.005282651633024216, 'BQA': 2720, 'ID': 1, 'NDVI': 0.8285050988197327, 'date': 20210105, 'doy': 5, 'month': 1, 'random': 0.9401598291236601}}


In [38]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [39]:
# Clasificando con las mismas bandas de entrenamiento
result = greenest.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classificado')
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [40]:
result.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [41]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [42]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [43]:
land = result.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [44]:
Map.addLayer(landcoverf, {}, 'Land cover Random Forest Región')
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [45]:
train_accuracy = classifier.confusionMatrix()

In [46]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 618, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 [0, 0, 807, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 3026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 8, 1585, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 1, 0, 0, 1, 937, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 621, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 6, 0, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 82, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 226, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 158, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3132, 0, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 240, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 789, 0, 0, 0, 0],
 [0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 204, 0, 0, 0],
 [0, 0, 0,

In [47]:
train_accuracy.accuracy().getInfo()

0.9949448852067683

In [48]:
train_accuracy.kappa().getInfo()

0.9942068120574772

In [49]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9935691318327974],
 [0.9987623762376238],
 [1],
 [0.9937304075235109],
 [1],
 [0.9925847457627118],
 [0.9983922829581994],
 [0.8636363636363636],
 [1],
 [1],
 [0.9937106918238994],
 [0.999680817108203],
 [0.97165991902834],
 [0.9924528301886792],
 [0.9714285714285714],
 [0.996875],
 [0.9107142857142857],
 [0.9939320388349514]]

In [50]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9935691318327974,
  0.992619926199262,
  0.9927821522309711,
  0.9987397605545053,
  0.98989898989899,
  0.9904862579281184,
  0.9920127795527156,
  1,
  1,
  0.9912280701754386,
  0.9937106918238994,
  0.9993618379068283,
  0.9917355371900827,
  0.98625,
  1,
  0.9953198127925117,
  1,
  0.997563946406821]]

In [51]:
validacion = datasetval.classify(classifier)

NameError: name 'datasetval' is not defined

In [ ]:
testAccuracy = validacion.errorMatrix('ID', 'classification')

In [ ]:
testAccuracy.accuracy().getInfo()

In [ ]:
testAccuracy.getInfo()